In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
file_path = '/content/drive/MyDrive/Cherry Blossom/DATA/washingtondc.csv'
df_bloom = pd.read_csv(file_path)
df_bloom.rename(columns={'bloom_date': 'DATE'}, inplace=True)
df_bloom["DATE"] = pd.to_datetime(df_bloom["DATE"], errors="coerce")

In [ ]:
df_bloom

,location,lat,long,alt,year,DATE,bloom_doy
0,washingtondc,38.88535,-77.038628,0,1921,1921-03-20,79
1,washingtondc,38.88535,-77.038628,0,1922,1922-04-07,97
2,washingtondc,38.88535,-77.038628,0,1923,1923-04-09,99
3,washingtondc,38.88535,-77.038628,0,1924,1924-04-13,104
4,washingtondc,38.88535,-77.038628,0,1925,1925-03-27,86
...,...,...,...,...,...,...,...
99,washingtondc,38.88535,-77.038628,0,2020,2020-03-20,80
100,washingtondc,38.88535,-77.038628,0,2021,2021-03-28,87
101,washingtondc,38.88535,-77.038628,0,2022,2022-03-21,80
102,washingtondc,38.88535,-77.038628,0,2023,2023-03-23,82


In [ ]:
file_path = '/content/drive/MyDrive/Cherry Blossom/DATA/washingtondc_weather.csv'
df_weather = pd.read_csv(file_path)

In [ ]:
def calculate_tavg(x, y, eta):
  return eta * x + (1 - eta) * y

df_weather["TAVG"] = df_weather.apply(lambda x: calculate_tavg(x['TMAX'], x['TMIN'], 0.5), axis=1)

df_weather["DATE"] = pd.to_datetime(df_weather["DATE"], errors="coerce")
df_weather["year"] = df_weather["DATE"].dt.year

df_weather["DOY"] = df_weather["DATE"].dt.dayofyear

df_weather = df_weather.merge(df_bloom[["year", "bloom_doy"]], on="year", how="left")

df_weather_2025 = df_weather[df_weather["year"] == 2025]
df_weather = df_weather[df_weather["year"] != 2025]

In [ ]:
df_weather_avaliable = df_weather[df_weather["DOY"] <= df_weather["bloom_doy"]]

BASE_TEMP = 5

# df_weather_avaliable["GDD"] = np.maximum(df_weather_avaliable["TAVG"] - BASE_TEMP, 0)
df_weather_avaliable["GDD"] = df_weather_avaliable["TAVG"] - BASE_TEMP
df_weather_avaliable = df_weather_avaliable.sort_values(by=["year", "DOY"]).reset_index(drop=True)
df_weather_avaliable["GDD_cumsum"] = df_weather_avaliable.groupby("year")["GDD"].cumsum()
df_weather_avaliable["PRCP_cumsum"] = df_weather_avaliable.groupby("year")["PRCP"].cumsum()

df_weather_avaliable

<ipython-input-77-a0af7d45b101>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather_avaliable["GDD"] = df_weather_avaliable["TAVG"] - BASE_TEMP


,FMTM,WSF2,WSF5,SNOW,PRCP,WT08,DATE,WDF2,AWND,WDF5,...,AWBT,ADPT,WT07,TAVG,year,DOY,bloom_doy,GDD,GDD_cumsum,PRCP_cumsum
0,1812.0,67.0,72.0,0,0,1.0,2000-01-01,160.0,19.0,150.0,...,NaN,NaN,NaN,6.95,2000,1,77.0,1.95,1.95,0
1,1453.0,112.0,130.0,0,0,1.0,2000-01-02,200.0,57.0,190.0,...,NaN,NaN,NaN,13.35,2000,2,77.0,8.35,10.30,0
2,2345.0,94.0,103.0,0,0,NaN,2000-01-03,170.0,49.0,170.0,...,NaN,NaN,NaN,15.55,2000,3,77.0,10.55,20.85,0
3,1113.0,148.0,183.0,0,310,NaN,2000-01-04,190.0,71.0,190.0,...,NaN,NaN,NaN,15.30,2000,4,77.0,10.30,31.15,310
4,808.0,125.0,152.0,0,0,NaN,2000-01-05,310.0,63.0,310.0,...,NaN,NaN,NaN,5.30,2000,5,77.0,0.30,31.45,310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2217,NaN,63.0,94.0,0,0,NaN,2024-03-13,230.0,19.0,220.0,...,89.0,28.0,NaN,15.85,2024,73,77.0,10.85,119.15,2459
2218,NaN,67.0,89.0,0,0,NaN,2024-03-14,180.0,31.0,180.0,...,106.0,44.0,NaN,16.65,2024,74,77.0,11.65,130.80,2459
2219,NaN,98.0,125.0,0,5,NaN,2024-03-15,340.0,40.0,340.0,...,133.0,89.0,NaN,19.15,2024,75,77.0,14.15,144.95,2464
2220,NaN,76.0,98.0,0,0,NaN,2024-03-16,170.0,40.0,170.0,...,89.0,33.0,NaN,13.90,2024,76,77.0,8.90,153.85,2464


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

FEATURES = ["GDD_cumsum", "TMAX", "TMIN", "TAVG", "PRCP_cumsum"] #
TARGET = "bloom_doy"

df_final = df_weather_avaliable[
    (df_weather_avaliable["DOY"] >= df_weather_avaliable["bloom_doy"] - 3) &
    (df_weather_avaliable["DOY"] <= df_weather_avaliable["bloom_doy"] + 3)
]

X = df_final[FEATURES]
y = df_final[TARGET]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
print("Filtered df_final shape:", df_final.shape)

Filtered df_final shape: (100, 48)


In [ ]:
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_test shape: (20, 5)
y_test shape: (20,)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=3),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=2, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=20, random_state=42),
    "Ridge Regression": Ridge(alpha=1.0),
    "Bayesian Ridge": BayesianRidge(),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=3),
    "SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "Lasso Regression": Lasso(alpha=0.01)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R²": r2}

df_results = pd.DataFrame(results).T
print(df_results)

                         MSE        R²
Linear Regression  30.856903  0.176930
Decision Tree      24.285403  0.352217
Random Forest      25.903136  0.309065
XGBoost            21.979961  0.413711
Ridge Regression   30.730680  0.180297
Bayesian Ridge     30.133607  0.196223
KNN Regressor      31.322222  0.164518
SVR                27.328217  0.271053
Lasso Regression   30.814492  0.178061


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

df_weather_available = df_weather.copy()

BASE_TEMP = 5
df_weather_available["GDD"] = np.maximum(df_weather_available["TAVG"] - BASE_TEMP, 0)
df_weather_available["GDD_cumsum"] = df_weather_available.groupby("year")["GDD"].cumsum()

df_weather_available["PRCP_cumsum"] = df_weather_available.groupby("year")["PRCP"].cumsum()

df_weather_available["TMAX_TMIN_DIFF"] = df_weather_available["TMAX"] - df_weather_available["TMIN"]
df_weather_available["AVG_GDD_CUMSUM"] = df_weather_available["GDD_cumsum"] / df_weather_available["DOY"]
df_weather_available["PRCP_LAG1"] = df_weather_available["PRCP_cumsum"].shift(1).fillna(0)
df_weather_available["GDD_DIFF"] = df_weather_available["GDD_cumsum"].diff().fillna(0)

df_final = df_weather_available[
    (df_weather_available["DOY"] >= df_weather_available["bloom_doy"] - 3) &
    (df_weather_available["DOY"] <= df_weather_available["bloom_doy"] + 3)
]

FEATURES = ["GDD_cumsum", "TMAX", "TMIN", "TAVG", "PRCP_cumsum",
            "TMAX_TMIN_DIFF", "AVG_GDD_CUMSUM", "PRCP_LAG1", "GDD_DIFF"]
TARGET = "bloom_doy"

X = df_final[FEATURES]
y = df_final[TARGET]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(max_depth=3),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=2, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=20, random_state=42),
    "Ridge Regression": Ridge(alpha=1.0),
    "Bayesian Ridge": BayesianRidge(),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=3),
    "SVR": SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "Lasso Regression": Lasso(alpha=0.01)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "R²": r2}

df_results = pd.DataFrame(results).T
print(df_results)

                         MSE        R²
Linear Regression   4.471300  0.893146
Decision Tree      16.731539  0.600153
Random Forest      12.647052  0.697764
XGBoost             3.281502  0.921579
Ridge Regression    4.593370  0.890229
Bayesian Ridge      4.556172  0.891118
KNN Regressor      13.368254  0.680528
SVR                18.212762  0.564755
Lasso Regression    4.515769  0.892083


In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
# from tensorflow.keras.optimizers import Adam
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import train_test_split

# df_weather_available["TAVG_rolling"] = df_weather_available.groupby("year")["TAVG"].transform(lambda x: x.rolling(7, min_periods=1).mean())
# df_weather_available["TAVG_anomaly"] = df_weather_available["TAVG"] - df_weather_available.groupby("year")["TAVG"].transform("mean")
# df_weather_available["GDD_anomaly"] = df_weather_available["GDD_cumsum"] - df_weather_available.groupby("year")["GDD_cumsum"].transform("mean")

# FEATURES = ["GDD_cumsum", "TMAX", "TMIN", "TAVG", "PRCP_cumsum", "TAVG_rolling", "TAVG_anomaly", "GDD_anomaly"]
# TARGET = "bloom_doy"

# X = df_weather_available[FEATURES].values
# y = df_weather_available[TARGET].values

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# sequence_length = 15
# X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1]))

# X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# X_train_seq = np.array([X_train[i - sequence_length:i] for i in range(sequence_length, len(X_train))])
# y_train_seq = np.array(y_train[sequence_length:])

# X_test_seq = np.array([X_test[i - sequence_length:i] for i in range(sequence_length, len(X_test))])
# y_test_seq = np.array(y_test[sequence_length:])

# X_train_seq = X_train_seq.reshape(X_train_seq.shape[0], sequence_length, X_train.shape[1])
# X_test_seq = X_test_seq.reshape(X_test_seq.shape[0], sequence_length, X_test.shape[1])

# model = Sequential([
#     Bidirectional(LSTM(128, return_sequences=True, input_shape=(sequence_length, X_train.shape[1]))),
#     Dropout(0.2),
#     Bidirectional(LSTM(64, return_sequences=True)),
#     Dropout(0.2),
#     LSTM(32, return_sequences=False),
#     Dense(16),
#     Dense(1)
# ])

# optimizer = Adam(learning_rate=0.0005)
# model.compile(optimizer=optimizer, loss="mse")

# history = model.fit(X_train_seq, y_train_seq, validation_data=(X_test_seq, y_test_seq), epochs=100, verbose=1)

# y_pred = model.predict(X_test_seq)

# mse = mean_squared_error(y_test_seq, y_pred)
# r2 = r2_score(y_test_seq, y_pred)

# print(f"LSTM Model Performance:")
# print(f"Mean Squared Error (MSE): {mse:.4f}")
# print(f"R² Score: {r2:.4f}")

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Train the ARIMA model using only TAVG (average air temperature) from 2020-2024
df_train = df_weather[df_weather["year"] < 2025][["DATE", "TAVG"]]
df_train.set_index("DATE", inplace=True)

model = ARIMA(df_train, order=(5,1,0))  # ARIMA(5,1,0) represents autoregression + difference
model_fit = model.fit()

# Predicted temperatures from February to April 2025
future_dates = pd.date_range("2025-02-20", "2025-04-30")
forecast = model_fit.forecast(steps=len(future_dates))

df_forecast = pd.DataFrame({"DATE": future_dates, "TAVG": forecast})
df_forecast["year"] = 2025
df_forecast["DOY"] = df_forecast["DATE"].dt.dayofyear

df_weather = pd.concat([df_weather, df_forecast], ignore_index=True)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency -1D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency -1D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inf

In [ ]:
year = 2025

df_test = df_weather[df_weather["year"] == year].copy()
df_test.fillna(0, inplace=True)

# Set base temperature (usually 5°C)
BASE_TEMP = 5

# Calculate Daily GDD
df_test["GDD"] = np.maximum(df_test["TAVG"] - BASE_TEMP, 0)
df_test["GDD_cumsum"] = df_test.groupby("year")["GDD"].cumsum()
df_test["PRCP_cumsum"] = df_test.groupby("year")["PRCP"].cumsum()
df_test["TMAX_TMIN_DIFF"] = df_test["TMAX"] - df_test["TMIN"]
df_test["AVG_GDD_CUMSUM"] = df_test["GDD_cumsum"].rolling(window=7, min_periods=1).mean()
df_test["PRCP_LAG1"] = df_test["PRCP"].shift(1).fillna(0)
df_test["GDD_DIFF"] = df_test["GDD"].diff().fillna(0)


latest_data = df_test.iloc[-1][FEATURES].values.reshape(1, -1)

latest_data_scaled = scaler.transform(latest_data)

best_model = models["Random Forest"]  # Change to the model with the highest R²

predicted_bloom_doy = best_model.predict(latest_data_scaled)[0]

print(f" Predict the cherry blossom blooming date in {year} (DOY): {int(predicted_bloom_doy)}")

In [ ]:
#Refine Swiss Prediction
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load datasets
nyc_data = pd.read_csv("/mnt/data/nyc.csv")
kyoto_data = pd.read_csv("/mnt/data/Kyoto_weather.csv")
liestal_data = pd.read_csv("/mnt/data/liestal.csv")
liestal_weather = pd.read_csv("/mnt/data/liestal_weather.csv")

# Data preprocessing
# Extract relevant features from Kyoto and Liestal data
df_kyoto = kyoto_data[['year', 'bloom_doy']].dropna()
df_liestal = liestal_data[['year', 'bloom_doy']].dropna()

# Merge Liestal bloom data with weather data
liestal_weather['DATE'] = pd.to_datetime(liestal_weather['DATE'])
liestal_weather['year'] = liestal_weather['DATE'].dt.year
liestal_weather_avg = liestal_weather.groupby('year')[['TMAX', 'TMIN', 'PRCP']].mean().reset_index()
df_liestal = df_liestal.merge(liestal_weather_avg, on='year', how='left')

# Train-test split
X_liestal = df_liestal[['year', 'TMAX', 'TMIN', 'PRCP']]
y_liestal = df_liestal['bloom_doy']
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_liestal, y_liestal, test_size=0.2, random_state=42)

# Train multiple models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42)
}

best_model = None
best_score = float('inf')

for name, model in models.items():
    model.fit(X_train_l, y_train_l)
    y_pred = model.predict(X_test_l)
    mse = mean_squared_error(y_test_l, y_pred)
    print(f"{name} - MSE: {mse:.2f}, R²: {r2_score(y_test_l, y_pred):.2f}")
    if mse < best_score:
        best_score = mse
        best_model = model

# Predict bloom date for Liestal-Weideli (2025)
liestal_pred_2025 = best_model.predict([[2025, liestal_weather_avg['TMAX'].mean(), liestal_weather_avg['TMIN'].mean(), liestal_weather_avg['PRCP'].mean()]])[0]
print(f"Predicted Liestal-Weideli Bloom Day of Year: {int(liestal_pred_2025)}")



#Import NYC city data and use reference from Mational Park Service 
import numpy as np

# Ensure input is a NumPy array
nyc_pred_2025_doy = int(xgb_model_nyc.predict(np.array([[2025]]))[0])

# Convert DOY to actual date in 2025
predicted_bloom_date_2025 = pd.to_datetime(f"2025-{nyc_pred_2025_doy}", format="%Y-%j").strftime("%B %d")

predicted_bloom_date_2025


